# Deploy

In [19]:
# files
import os

# images
import cv2 as cv

# pandas
import pandas as pd

# load model
from tensorflow.keras.models import load_model

In [20]:
# utility.py functions
from utility import predict_image

### Load pretrained model

In [21]:
autoencoder = load_model("../Models/autoencoderFC")

### Save anomaly location

In [22]:
dir = "../Data/Inference"

In [23]:
imgs_anom, imgs_name, imgs_score, imgs_valid = [], [], [], []
formats = ['jpg', 'jpeg', 'png', 'bmp']

for img in os.listdir(dir):
    if img[-3:] in formats:
        image = cv.imread((dir + '/' + img), cv.IMREAD_GRAYSCALE)
        anom, img_anom, score = predict_image(image, autoencoder)
        imgs_name.append(img)
        imgs_anom.append(anom)
        imgs_score.append(score)
        if score < 0.75:
            imgs_valid.append("Non valid prediction")
        else:
            imgs_valid.append("Valid prediction")
            
        cv.imwrite(dir + "/Predictions/pred_" + img , img_anom*255)

### Save prediction

In [24]:
df = {"name_img": imgs_name,
      "anomaly": imgs_anom,
      "reconstruction_score": imgs_score,
      "threshold": imgs_valid}

In [25]:
df = pd.DataFrame(df, index=None)
df.to_csv(dir + "/Predictions/summary.csv")
df.head()

,name_img,anomaly,reconstruction_score,threshold
0,NOK_2.jpg,True,0.869532,Valid prediction
1,NOK_7.jpg,True,0.957857,Valid prediction
2,OK_1.jpg,False,0.959577,Valid prediction
